In [1]:
import os
import sys
import glob
import argparse
from collections import Counter, defaultdict
from typing import Iterator, Tuple, Union, List
from multiprocessing import Pool, cpu_count
from collections import Counter
import pandas as pd
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
    StratifiedKFold,
    cross_val_score,
    train_test_split,
    RandomizedSearchCV,
)
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

from xgboost import XGBClassifier
import optuna


In [2]:

AA_ALPHABET = list("ACDEFGHIKLMNPQRSTVWY")


def extract_kmers(seq, k=3, alphabet=AA_ALPHABET):
    if not isinstance(seq, str):
        return []
    seq = seq.strip()
    n = len(seq)
    if n < k:
        return []
    out = []
    for i in range(n - k + 1):
        kmer = seq[i:i + k]
        if all(c in alphabet for c in kmer):
            out.append(kmer)
    return out



In [3]:
def convert_to_top_seq_format(top_df, dataset_name, default_prob=-999.0):
    df = top_df.copy().reset_index(drop=True)

    n = len(df)
    df.insert(0, "ID", [f"{dataset_name}_seq_top_{i+1}" for i in range(n)])
    df.insert(1, "dataset", dataset_name)
    df.insert(2, "label_positive_probability", float(default_prob))

    cols = [
        "ID",
        "dataset",
        "label_positive_probability",
        "junction_aa",
        "v_call",
        "j_call",
    ]
    # keep importance_score (and any other extra cols) at the end
    cols = cols + [c for c in df.columns if c not in cols]
    return df[cols]
def to_submission_format(test_pred_df, dataset_name="test_dataset_1"):
    df = test_pred_df.copy()

    df["ID"] = df["repertoire_id"]
    df["dataset"] = dataset_name
    df["label_positive_probability"] = df["prediction"]

    df["junction_aa"] = -999.0
    df["v_call"] = -999.0
    df["j_call"] = -999.0

    cols = [
        "ID",
        "dataset",
        "label_positive_probability",
        "junction_aa",
        "v_call",
        "j_call",
    ]
    return df[cols]

In [4]:
def load_data_generator(data_dir: str, metadata_filename='metadata.csv') -> Iterator[
    Union[Tuple[str, pd.DataFrame, bool], Tuple[str, pd.DataFrame]]]:
    """
    A generator to load immune repertoire data.

    This function operates in two modes:
    1.  If metadata is found, it yields data based on the metadata file.
    2.  If metadata is NOT found, it uses glob to find and yield all '.tsv'
        files in the directory.

    Args:
        data_dir (str): The path to the directory containing the data.

    Yields:
        An iterator of tuples. The format depends on the mode:
        - With metadata: (repertoire_id, pd.DataFrame, label_positive)
        - Without metadata: (filename, pd.DataFrame)
    """
    metadata_path = os.path.join(data_dir, metadata_filename)

    if os.path.exists(metadata_path):
        metadata_df = pd.read_csv(metadata_path)
        for row in metadata_df.itertuples(index=False):
            file_path = os.path.join(data_dir, row.filename)
            try:
                repertoire_df = pd.read_csv(file_path, sep='\t')
                yield row.repertoire_id, repertoire_df, row.label_positive
            except FileNotFoundError:
                print(f"Warning: File '{row.filename}' listed in metadata not found. Skipping.")
                continue
    else:
        search_pattern = os.path.join(data_dir, '*.tsv')
        tsv_files = glob.glob(search_pattern)
        for file_path in sorted(tsv_files):
            try:
                filename = os.path.basename(file_path)
                repertoire_df = pd.read_csv(file_path, sep='\t')
                yield filename, repertoire_df
            except Exception as e:
                print(f"Warning: Could not read file '{file_path}'. Error: {e}. Skipping.")
                continue


def load_full_dataset(data_dir: str) -> pd.DataFrame:
    """
    Loads all TSV files from a directory and concatenates them into a single DataFrame.

    This function handles two scenarios:
    1. If metadata.csv exists, it loads data based on the metadata and adds
       'repertoire_id' and 'label_positive' columns.
    2. If metadata.csv does not exist, it loads all .tsv files and adds
       a 'filename' column as an identifier.

    Args:
        data_dir (str): The path to the data directory.

    Returns:
        pd.DataFrame: A single, concatenated DataFrame containing all the data.
    """
    metadata_path = os.path.join(data_dir, 'metadata.csv')
    df_list = []
    data_loader = load_data_generator(data_dir=data_dir)

    if os.path.exists(metadata_path):
        metadata_df = pd.read_csv(metadata_path)
        total_files = len(metadata_df)
        for rep_id, data_df, label in tqdm(data_loader, total=total_files, desc="Loading files"):
            data_df['ID'] = rep_id
            data_df['label_positive'] = label
            df_list.append(data_df)
    else:
        search_pattern = os.path.join(data_dir, '*.tsv')
        total_files = len(glob.glob(search_pattern))
        for filename, data_df in tqdm(data_loader, total=total_files, desc="Loading files"):
            data_df['ID'] = os.path.basename(filename).replace(".tsv", "")
            df_list.append(data_df)

    if not df_list:
        print("Warning: No data files were loaded.")
        return pd.DataFrame()

    full_dataset_df = pd.concat(df_list, ignore_index=True)
    return full_dataset_df

def mismatched_neighbors(kmer, alphabet=AA_ALPHABET, max_mismatches=1, include_self=True):
    k = len(kmer)
    if max_mismatches == 0:
        return [kmer] if include_self else []
    neighbors = set()
    if include_self:
        neighbors.add(kmer)
    for pos in range(k):
        for aa in alphabet:
            if aa == kmer[pos]:
                continue
            new_kmer = kmer[:pos] + aa + kmer[pos + 1 :]
            neighbors.add(new_kmer)
    return list(neighbors)


def mismatch_smooth_counts(counts, k=3, alphabet=AA_ALPHABET):
    out = Counter()
    for kmer, c in counts.items():
        if len(kmer) != k:
            continue
        for nb in mismatched_neighbors(kmer, alphabet=alphabet, max_mismatches=1, include_self=True):
            out[nb] += c
    return out


In [5]:
AA_ALPHABET = list("ACDEFGHIKLMNPQRSTVWY")


def extract_gapped_trimers(seq, patterns=((1,0), (0,1),
    (2,0), (0,2), (1,1),
    (3,0), (0,3), (2,1), (1,2)), alphabet=AA_ALPHABET):
    if not isinstance(seq, str):
        return []
    seq = seq.strip()
    n = len(seq)
    out = []
    for gap1, gap2 in patterns:
        window = 3 + gap1 + gap2
        if n < window:
            continue
        for i in range(n - window + 1):
            a = seq[i]
            b = seq[i + 1 + gap1]
            c = seq[i + 2 + gap1 + gap2]
            if a in alphabet and b in alphabet and c in alphabet:
                key = f"{a}{b}{c}|g{gap1}{gap2}"
                out.append(key)
    return out


def encode_repertoire(
    seqs,
    k=3,
    use_gaps=False,
    use_mismatch=False,
    gap_patterns=((1,0), (0,1),
    (2,0), (0,2), (1,1),
    (3,0), (0,3), (2,1), (1,2)),
    alphabet=AA_ALPHABET,
):
    exact_counts = Counter()
    gap_counts = Counter()

    for s in seqs:
        if not isinstance(s, str):
            continue
        exact_counts.update(extract_kmers(s, k=k, alphabet=alphabet))
        if use_gaps and k == 3:
            gap_counts.update(
                extract_gapped_trimers(
                    s,
                    patterns=gap_patterns,
                    alphabet=alphabet,
                )
            )

    if use_mismatch:
        mm_counts = mismatch_smooth_counts(exact_counts, k=k, alphabet=alphabet)
    else:
        mm_counts = Counter()

    features = {}
    for kmer, c in exact_counts.items():
        features[f"exact_{kmer}"] = c
    for kmer, c in gap_counts.items():
        features[f"gap_{kmer}"] = c
    for kmer, c in mm_counts.items():
        features[f"mm1_{kmer}"] = c

    return features

def load_and_encode_repertoires_advanced(
    data_dir,
    k=3,
    use_gaps=False,
    use_mismatch=False,
    metadata_filename="metadata.csv",
    n_jobs=None,
):
    metadata_path = os.path.join(data_dir, metadata_filename)
    use_metadata = os.path.exists(metadata_path)

    tasks = []

    if use_metadata:
        metadata_df = pd.read_csv(metadata_path)
        for row in metadata_df.itertuples(index=False):
            path = os.path.join(data_dir, row.filename)
            tasks.append((row.repertoire_id, path, row.label_positive, k, use_gaps, use_mismatch))
    else:
        pattern = os.path.join(data_dir, "*.tsv")
        files = sorted(glob.glob(pattern))
        for path in files:
            rep_id = os.path.basename(path).replace(".tsv", "")
            tasks.append((rep_id, path, None, k, use_gaps, use_mismatch))

    total = len(tasks)
    if total == 0:
        return pd.DataFrame(), pd.DataFrame()

    if n_jobs is None or n_jobs < 1:
        n_jobs = cpu_count()

    feature_records = []
    meta_records = []

    with Pool(processes=n_jobs) as pool:
        for feats, meta in tqdm(
            pool.imap(_encode_one_repertoire, tasks),
            total=total,
            desc=f"Encoding k={k} advanced",
        ):
            feature_records.append(feats)
            meta_records.append(meta)

    X = pd.DataFrame(feature_records).fillna(0).set_index("ID")
    meta_df = pd.DataFrame(meta_records)
    return X, meta_df

def _encode_one_repertoire(args):
    rep_id, path, label, k, use_gaps, use_mismatch = args
    df = pd.read_csv(path, sep="\t")
    seqs = df["junction_aa"].dropna().astype(str).tolist()
    feats = encode_repertoire(
        seqs,
        k=k,
        use_gaps=use_gaps,
        use_mismatch=use_mismatch,
    )
    feats["ID"] = rep_id

    meta = {"ID": rep_id}
    if label is not None:
        meta["label_positive"] = label

    return feats, meta




In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def fit_xgb_no_leakage_with_importance(
    X,
    y,
    n_top_features=200,
    random_state=123,
    n_iter=100,
    n_jobs=-1,
):
    y_arr = np.asarray(y).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y_arr,
        test_size=0.2,
        stratify=y_arr,
        random_state=random_state,
    )

    model_full, cv_scores_full, importance_full = fit_xgb(
        X_train,
        y_train,
        random_state=random_state,
        n_iter=n_iter,
        n_jobs=n_jobs,
    )

    top_features = importance_full["feature"].head(n_top_features).tolist()

    X_train_top = X_train.loc[:, top_features]
    X_test_top = X_test.loc[:, top_features]

    model_top, cv_scores_top, importance_top = fit_xgb(
        X_train_top,
        y_train,
        random_state=random_state,
        n_iter=n_iter,
        n_jobs=n_jobs,
    )

    y_test_pred = model_top.predict_proba(X_test_top)[:, 1]
    test_auc = roc_auc_score(y_test, y_test_pred)

    return {
        "initial_model": model_full,
        "initial_cv_scores": cv_scores_full,
        "initial_importance": importance_full,
        "selected_features": top_features,
        "final_model": model_top,
        "final_cv_scores": cv_scores_top,
        "final_importance": importance_top,
        "test_auc": test_auc,
    }

In [7]:
def fit_xgb(X, y, random_state=123, n_iter=150, n_jobs=-1):
    y_arr = np.asarray(y).astype(int)

    cv = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=random_state,
    )

    max_depth_values = np.arange(3, 8)
    n_estimators_values = np.linspace(200, 1000, 9, dtype=int)
    learning_rate_values = np.logspace(-3, -0.7, 8)
    subsample_values = np.linspace(0.6, 1.0, 5)
    colsample_bytree_values = np.linspace(0.6, 1.0, 5)
    min_child_weight_values = [3, 5, 7]
    gamma_values = [0.0, 0.1, 0.2, 0.5]
    reg_lambda_values = np.logspace(-1, 2, 6)
    reg_alpha_values = [0.0, 1e-3, 1e-2, 1e-1, 1.0, 10.0]

    fixed_params = dict(
        objective="binary:logistic",
        eval_metric="auc",
        tree_method="hist",
        random_state=random_state,
        n_jobs=-1,
        device="cpu",
    )

    def objective(trial):
        params = {
            "max_depth": trial.suggest_categorical("max_depth", list(max_depth_values)),
            "n_estimators": trial.suggest_categorical("n_estimators", list(n_estimators_values)),
            "learning_rate": trial.suggest_categorical("learning_rate", list(learning_rate_values)),
            "subsample": trial.suggest_categorical("subsample", list(subsample_values)),
            "colsample_bytree": trial.suggest_categorical("colsample_bytree", list(colsample_bytree_values)),
            "min_child_weight": trial.suggest_categorical("min_child_weight", min_child_weight_values),
            "gamma": trial.suggest_categorical("gamma", gamma_values),
            "reg_lambda": trial.suggest_categorical("reg_lambda", list(reg_lambda_values)),
            "reg_alpha": trial.suggest_categorical("reg_alpha", reg_alpha_values),
        }
        model = XGBClassifier(**fixed_params, **params)
        scores = cross_val_score(
            model,
            X.values,
            y_arr,
            cv=cv,
            scoring="roc_auc",
            n_jobs=n_jobs,
        )
        return scores.mean()

    sampler = optuna.samplers.TPESampler(seed=random_state)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=n_iter, n_jobs=7,    show_progress_bar=True)

    best_params = study.best_params
    best_model = XGBClassifier(**fixed_params, **best_params)
    best_model.fit(X.values, y_arr)

    scores = cross_val_score(
        best_model,
        X.values,
        y_arr,
        cv=cv,
        scoring="roc_auc",
        n_jobs=n_jobs,
    )

    importance = pd.DataFrame(
        {"feature": X.columns, "importance": best_model.feature_importances_}
    ).sort_values("importance", ascending=False)

    return best_model, scores, importance

In [8]:
def basic_kmer_feature_filter(X, min_nonzero_repertoires=5, min_total_count=10):
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    nonzero_counts = (X != 0).sum(axis=0)
    total_counts = X.sum(axis=0)

    mask = (nonzero_counts >= min_nonzero_repertoires) & (total_counts >= min_total_count)
    X_filtered = X.loc[:, mask]
    selected_features = X_filtered.columns.tolist()
    return X_filtered, selected_features


def apply_feature_filter(X, selected_features):
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    for f in selected_features:
        if f not in X.columns:
            X[f] = 0

    return X[selected_features]

In [9]:
def normalize_kmer_rows_by_category(X, col_categories=None):
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    if col_categories is None:
        cats = []
        for c in X.columns:
            if "_" in c:
                prefix = c.split("_", 1)[0]
                if prefix == "gap" and "|g" in c:
                    code = c.split("|g", 1)[1]
                    cat = f"gap_g{code}"
                else:
                    cat = prefix
            else:
                cat = "other"
            cats.append(cat)
        col_categories = pd.Index(cats)
    else:
        col_categories = pd.Index(col_categories)

    group_sums = X.groupby(col_categories, axis=1).transform("sum")
    X_norm = X.div(group_sums.replace(0, np.nan))
    return X_norm.fillna(0.0)

In [10]:
from collections import Counter
from multiprocessing import Pool, cpu_count
from functools import partial
from tqdm import tqdm
import pandas as pd


def _score_one_sequence(
    s,
    k,
    alphabet,
    gap_patterns,
    use_gaps,
    use_mismatch,
    importance_map,
):
    s = str(s).strip()
    if len(s) < k:
        return 0.0

    exact_kmers = extract_kmers(s, k=k, alphabet=alphabet)
    exact_counts = Counter(exact_kmers)

    score = 0.0
    for kmer, cnt in exact_counts.items():
        feat_name = f"exact_{kmer}"
        imp = importance_map.get(feat_name)
        if imp is not None:
            score += imp * cnt

    if use_gaps and k == 3:
        gapped = extract_gapped_trimers(s, patterns=gap_patterns, alphabet=alphabet)
        gap_counts = Counter(gapped)
        for gk, cnt in gap_counts.items():
            feat_name = f"gap_{gk}"
            imp = importance_map.get(feat_name)
            if imp is not None:
                score += imp * cnt

    if use_mismatch:
        mm_counts = mismatch_smooth_counts(exact_counts, k=k, alphabet=alphabet)
        for mk, cnt in mm_counts.items():
            feat_name = f"mm1_{mk}"
            imp = importance_map.get(feat_name)
            if imp is not None:
                score += imp * cnt

    return score


def score_sequences_by_kmer_importance(
    sequences_df: pd.DataFrame,
    importance_df: pd.DataFrame,
    sequence_col: str = "junction_aa",
    alphabet=AA_ALPHABET,
    gap_patterns=(
        (1, 0),
        (0, 1),
        (2, 0),
        (0, 2),
        (1, 1),
        (3, 0),
        (0, 3),
        (2, 1),
        (1, 2),
    ),
    use_gaps: bool = True,
    use_mismatch: bool = True,
    n_jobs: int = None,
) -> pd.DataFrame:
    if sequence_col not in sequences_df.columns:
        raise KeyError(sequence_col)

    feat_names = importance_df["feature"].astype(str).tolist()

    exact_feats = [f for f in feat_names if f.startswith("exact_")]
    k_values = set()

    if exact_feats:
        k_values = {len(f.split("exact_", 1)[1]) for f in exact_feats}
    else:
        mm_feats = [f for f in feat_names if f.startswith("mm1_")]
        if mm_feats:
            k_values = {len(f.split("mm1_", 1)[1]) for f in mm_feats}
        else:
            gap_feats = [f for f in feat_names if f.startswith("gap_")]
            if gap_feats:
                tmp = set()
                for f in gap_feats:
                    rest = f.split("gap_", 1)[1]
                    core = rest.split("|g", 1)[0]
                    tmp.add(len(core))
                k_values = tmp

    if not k_values:
        raise ValueError("Cannot infer k from feature names; expected 'exact_', 'mm1_', or 'gap_' features")

    if len(k_values) != 1:
        raise ValueError("Inconsistent k across features")

    k = k_values.pop()

    importance_map = dict(zip(importance_df["feature"], importance_df["importance"]))

    if use_gaps:
        has_gap_feats = any(f.startswith("gap_") for f in feat_names)
        use_gaps = has_gap_feats and (k == 3)

    if use_mismatch:
        has_mm_feats = any(f.startswith("mm1_") for f in feat_names)
        use_mismatch = has_mm_feats

    seq_series = sequences_df[sequence_col].fillna("").astype(str)
    n = len(seq_series)

    if n_jobs is None or n_jobs < 1:
        n_jobs = cpu_count()
    else:
        n_jobs = min(n_jobs, cpu_count())

    if n_jobs == 1:
        scores = [
            _score_one_sequence(
                s,
                k,
                alphabet,
                gap_patterns,
                use_gaps,
                use_mismatch,
                importance_map,
            )
            for s in tqdm(seq_series, desc="scoring sequences", total=n)
        ]
    else:
        worker = partial(
            _score_one_sequence,
            k=k,
            alphabet=alphabet,
            gap_patterns=gap_patterns,
            use_gaps=use_gaps,
            use_mismatch=use_mismatch,
            importance_map=importance_map,
        )
        chunksize = max(1, n // (n_jobs * 8))

        with Pool(processes=n_jobs) as pool:
            scores = list(
                tqdm(
                    pool.imap(worker, seq_series, chunksize=chunksize),
                    total=n,
                    desc="scoring sequences",
                )
            )

    out = sequences_df.copy()
    out["importance_score"] = scores
    return out

In [11]:
def get_dataset_pairs(train_dir: str, test_dir: str) -> List[Tuple[str, List[str]]]:
    """Returns list of (train_path, [test_paths]) tuples for dataset pairs."""
    test_groups = defaultdict(list)
    for test_name in sorted(os.listdir(test_dir)):
        if test_name.startswith("test_dataset_"):
            base_id = test_name.replace("test_dataset_", "").split("_")[0]
            test_groups[base_id].append(os.path.join(test_dir, test_name))

    pairs = []
    for train_name in sorted(os.listdir(train_dir)):
        if train_name.startswith("train_dataset_"):
            train_id = train_name.replace("train_dataset_", "")
            train_path = os.path.join(train_dir, train_name)
            pairs.append((train_path, test_groups.get(train_id, [])))

    return pairs

In [12]:
from pathlib import Path

def load_and_encode_vj(folder_path: str, feature_colums=('v_call', 'j_call')):
    base_dir = Path(folder_path)
    dataset_name = base_dir.name

    dir_entries = list(base_dir.iterdir())
    tsv_list = [entry for entry in dir_entries if entry.suffix == '.tsv']
    non_tsv_names = [entry.name for entry in dir_entries if entry.suffix != '.tsv']
    print(f'Loading {len(tsv_list)} .tsv files from {dataset_name} (remaining: {non_tsv_names}).')

    meta_df = None
    meta_file = base_dir / 'metadata.csv'
    if meta_file.exists():
        meta_df = pd.read_csv(meta_file)
        meta_df.set_index('filename', inplace=True)

    records = []
    for tsv_file in tqdm(tsv_list, desc='Loading TSV files'):
        try:
            tab = pd.read_csv(tsv_file, sep='\t')
        except Exception as exc:
            print(f"Error loading {tsv_file.name}: {exc}")
            continue

        row_dict = {
            'ID': tsv_file.stem,
            'dataset': dataset_name,
        }

        if meta_df is not None and tsv_file.name in meta_df.index:
            row_dict['label_positive'] = int(meta_df.at[tsv_file.name, 'label_positive'])

        total_rows = len(tab)
        for feature in feature_colums:
            if feature not in tab.columns or total_rows == 0:
                continue
            freq_series = tab[feature].value_counts() / total_rows
            row_dict.update(freq_series.to_dict())

        records.append(row_dict)

    return pd.DataFrame(records).fillna(0)

In [13]:
import pandas as pd
from collections import defaultdict

def split_kmer_and_gap_families(X_train_norm, X_test_norm):
    def _subset(df, cols):
        if not cols:
            return pd.DataFrame(index=df.index)
        cols_existing = [c for c in cols if c in df.columns]
        if not cols_existing:
            return pd.DataFrame(index=df.index)
        return df.loc[:, cols_existing]

    # exact_* features
    exact_cols = [c for c in X_train_norm.columns if c.startswith("exact_")]
    X_train_exact = _subset(X_train_norm, exact_cols)
    X_test_exact = _subset(X_test_norm, exact_cols)

    # mm1_* features
    mm1_cols = [c for c in X_train_norm.columns if c.startswith("mm1_")]
    X_train_mm1 = _subset(X_train_norm, mm1_cols)
    X_test_mm1 = _subset(X_test_norm, mm1_cols)

    # gap_* features (all)
    all_gap_cols = sorted([c for c in X_train_norm.columns if c.startswith("gap_")])
    X_train_gap_all = _subset(X_train_norm, all_gap_cols)
    X_test_gap_all = _subset(X_test_norm, all_gap_cols)

    # split gap_* by pattern code, e.g. 'gap_AAA|g10' -> '10'
    def _get_gap_pattern(col):
        if not col.startswith("gap_"):
            return None
        rest = col.split("gap_", 1)[1]
        if "|g" not in rest:
            return None
        code = rest.split("|g", 1)[1]
        return code

    pattern_to_cols = defaultdict(list)
    for c in all_gap_cols:
        code = _get_gap_pattern(c) or "unknown"
        pattern_to_cols[code].append(c)

    X_train_gap_by_pattern = {}
    X_test_gap_by_pattern = {}
    for code, cols in pattern_to_cols.items():
        X_train_gap_by_pattern[code] = _subset(X_train_norm, cols)
        X_test_gap_by_pattern[code] = _subset(X_test_norm, cols)

    return {
        "exact": (X_train_exact, X_test_exact),
        "mm1": (X_train_mm1, X_test_mm1),
        "gap_all": (X_train_gap_all, X_test_gap_all),
        "gap_by_pattern": (X_train_gap_by_pattern, X_test_gap_by_pattern),
    }

In [14]:
import pandas as pd

def build_train_feature_blocks(X_train_norm, vj_train):
    parts = split_kmer_and_gap_families(X_train_norm, X_train_norm)

    X_train_exact, _ = parts["exact"]
    X_train_mm1, _   = parts["mm1"]

    gap_train_dict, _ = parts["gap_by_pattern"]

    X_train_vj = vj_train.reindex(X_train_norm.index).fillna(0.0)

    feature_blocks = {
        "exact": X_train_exact,
        "mm1":   X_train_mm1,
        "vj":    X_train_vj,
    }

    return feature_blocks, gap_train_dict

def train_base_models_for_current_dataset(
    X_train_norm,
    vj_train,
    y_train,
    random_state=123,
    n_iter=5,
    n_jobs=-1,
):
    feature_blocks, gap_train_dict = build_train_feature_blocks(X_train_norm, vj_train)
    print("splitting done")
    results = {}

    # exact, mm1, vj
    print("fitting exact, mm1, vj")

    for name, X_block in feature_blocks.items():
        if X_block is None or X_block.shape[1] == 0:
            continue
        model, scores, importance = fit_xgb(
            X=X_block,
            y=y_train,
            random_state=random_state,
            n_iter=n_iter,
            n_jobs=n_jobs,
        )
        results[name] = {
            "model": model,
            "cv_scores": scores,
            "importance": importance,
        }

    # individual gap families
    print("fitting individual gap families")
    for code, X_block in gap_train_dict.items():
        if X_block is None or X_block.shape[1] == 0:
            continue
        model, scores, importance = fit_xgb(
            X=X_block,
            y=y_train,
            random_state=random_state,
            n_iter=n_iter,
            n_jobs=n_jobs,
        )
        results[f"gap_g{code}"] = {
            "model": model,
            "cv_scores": scores,
            "importance": importance,
        }

    return results

In [15]:
def get_base_model_train_probs(X_train_norm, vj_train, base_results):
    feature_blocks, gap_train_dict = build_train_feature_blocks(X_train_norm, vj_train)

    blocks = {}
    for name, X_block in feature_blocks.items():
        if X_block is not None and X_block.shape[1] > 0 and name in base_results:
            blocks[name] = X_block

    for code, X_block in gap_train_dict.items():
        key = f"gap_g{code}"
        if X_block is not None and X_block.shape[1] > 0 and key in base_results:
            blocks[key] = X_block

    meta_X = pd.DataFrame(index=X_train_norm.index)

    for name, X_block in blocks.items():
        model = base_results[name]["model"]
        preds = model.predict_proba(X_block)[:, 1]
        meta_X[name] = preds

    return meta_X

def train_meta_model(meta_X_train, y_train, C=1.0):
    lr = LogisticRegression(
        penalty="l2",
        C=C,
        solver="lbfgs",
        max_iter=1000)

    lr.fit(meta_X_train.values, np.asarray(y_train).astype(int))
    return lr

def get_base_model_test_probs(X_train_norm, X_test_norm, vj_train, vj_test, base_results):
    parts = split_kmer_and_gap_families(X_train_norm, X_test_norm)

    _, X_test_exact = parts["exact"]
    _, X_test_mm1   = parts["mm1"]
    _, gap_test_dict = parts["gap_by_pattern"]

    X_test_vj = vj_test.reindex(X_test_norm.index).fillna(0.0)

    meta_X_test = pd.DataFrame(index=X_test_norm.index)

    for name, info in base_results.items():
        model = info["model"]

        if name == "exact":
            X_block = X_test_exact
        elif name == "mm1":
            X_block = X_test_mm1
        elif name == "vj":
            X_block = X_test_vj
        elif name.startswith("gap_g"):
            code = name[len("gap_g"):]
            X_block = gap_test_dict.get(code)
        else:
            continue

        if X_block is None or X_block.shape[1] == 0:
            continue

        preds = model.predict_proba(X_block)[:, 1]
        meta_X_test[name] = preds

    return meta_X_test

In [16]:
def _parse_gap_code(code: str):
    if not code.isdigit() or len(code) != 2:
        raise ValueError(f"bad gap code: {code}")
    return (int(code[0]), int(code[1]))


def compute_ensemble_sequence_scores(
    sequences_df: pd.DataFrame,
    base_results: dict,
    meta_model,
    meta_feature_names,
    alphabet=AA_ALPHABET,
    n_jobs: int = 1,
) -> pd.DataFrame:
    coef = meta_model.coef_[0]
    name_to_weight = {
        name: coef[i]
        for i, name in enumerate(meta_feature_names)
        if name in base_results
    }

    seq_scores = sequences_df.copy()
    per_model_cols = []

    for name, weight in name_to_weight.items():
        if abs(weight) < 1e-12:
            continue

        info = base_results[name]
        imp_df = info["importance"]

        if name == "exact":
            scored = score_sequences_by_kmer_importance(
                sequences_df,
                imp_df,
                sequence_col="junction_aa",
                alphabet=alphabet,
                use_gaps=False,
                use_mismatch=False,
                n_jobs=n_jobs,
            )
        elif name == "mm1":
            scored = score_sequences_by_kmer_importance(
                sequences_df,
                imp_df,
                sequence_col="junction_aa",
                alphabet=alphabet,
                use_gaps=False,
                use_mismatch=True,
                n_jobs=n_jobs,
            )
        elif name.startswith("gap_g"):
            code = name[len("gap_g"):]
            pattern = _parse_gap_code(code)
            scored = score_sequences_by_kmer_importance(
                sequences_df,
                imp_df,
                sequence_col="junction_aa",
                alphabet=alphabet,
                gap_patterns=(pattern,),
                use_gaps=True,
                use_mismatch=False,
                n_jobs=n_jobs,
            )
        else:
            continue

        col_name = f"score_{name}"
        seq_scores[col_name] = scored["importance_score"].values * weight
        per_model_cols.append(col_name)

    if per_model_cols:
        seq_scores["ensemble_score"] = seq_scores[per_model_cols].sum(axis=1)
    else:
        seq_scores["ensemble_score"] = 0.0

    return seq_scores
def get_top_sequences_ensemble(
    unique_seqs: pd.DataFrame,
    base_results: dict,
    meta_model,
    meta_feature_names,
    alphabet=AA_ALPHABET,
    n_jobs: int = 1,
    top_n: int = 50000,
) -> pd.DataFrame:
    scored = compute_ensemble_sequence_scores(
        sequences_df=unique_seqs,
        base_results=base_results,
        meta_model=meta_model,
        meta_feature_names=meta_feature_names,
        alphabet=alphabet,
        n_jobs=n_jobs,
    )
    scored_sorted = scored.sort_values("ensemble_score", ascending=False)
    top = scored_sorted.head(top_n)
    return top

In [17]:
#full_df = full_df.rename(columns={"ID": "repertoire_id", "label_positive": "label"})

In [18]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

def find_enriched_public_clones(
    df,
    min_pos_repertoires=3,
    min_log_odds=1.0,
    use_vj=True,
):
    df = df.copy()
    if use_vj:
        df["clone_id"] = (
            df["junction_aa"].astype("string")
            + "|" + df["v_call"].astype("string")
            + "|" + df["j_call"].astype("string")
        )
    else:
        df["clone_id"] = df["junction_aa"].astype("string")

    rep_label = df[["ID", "label_positive"]].drop_duplicates("ID")
    rep_label["label_positive"] = rep_label["label_positive"].astype(np.int8)

    n_pos = int((rep_label["label_positive"] == 1).sum())
    n_neg = int((rep_label["label_positive"] == 0).sum())

    tmp = df[["clone_id", "ID", "label_positive"]].drop_duplicates(["clone_id", "ID"])

    ct = pd.crosstab(tmp["clone_id"], tmp["label_positive"])
    ct = ct.reindex(columns=[0, 1], fill_value=0).astype(np.int32)
    ct = ct.rename(columns={0: "neg_reps", 1: "pos_reps"})

    alpha = 0.5
    pos = ct["pos_reps"].to_numpy()
    neg = ct["neg_reps"].to_numpy()

    log_odds = np.log(
        (pos + alpha) / (n_pos - pos + alpha)
    ) - np.log(
        (neg + alpha) / (n_neg - neg + alpha)
    )

    ct = ct.assign(log_odds=log_odds, clone_id=ct.index).reset_index(drop=True)

    enriched = ct[
        (ct["pos_reps"] >= min_pos_repertoires) &
        (ct["log_odds"] >= min_log_odds)
    ].reset_index(drop=True)

    return ct, enriched

def build_public_clone_presence_matrix(df, selected_clone_ids, use_vj=True):
    df = df.copy()
    if use_vj:
        df["clone_id"] = (
            df["junction_aa"].astype("string")
            + "|" + df["v_call"].astype("string")
            + "|" + df["j_call"].astype("string")
        )
    else:
        df["clone_id"] = df["junction_aa"].astype("string")

    df_sel = df[df["clone_id"].isin(selected_clone_ids)].copy()
    df_sel = df_sel[["ID", "clone_id"]].drop_duplicates()
    df_sel["value"] = 1.0

    X = (
        df_sel
        .pivot(index="ID", columns="clone_id", values="value")
        .reindex(columns=selected_clone_ids)
        .fillna(0.0)
        .astype(np.float32)
    )
    return X

def fit_l1_logit_cv(X, y, C=1.0, n_splits=5, random_state=0):
    X_values = X.values if hasattr(X, "values") else np.asarray(X)
    y_values = np.asarray(y).astype(int)

    skf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=random_state,
    )

    oof = np.zeros(len(y_values), dtype=float)

    for train_idx, val_idx in skf.split(X_values, y_values):
        clf = LogisticRegression(
            penalty="l1",
            C=C,
            solver="liblinear",
            max_iter=2000,
        )
        clf.fit(X_values[train_idx], y_values[train_idx])
        oof[val_idx] = clf.predict_proba(X_values[val_idx])[:, 1]

    auc = roc_auc_score(y_values, oof)

    final_clf = LogisticRegression(
        penalty="l1",
        C=C,
        solver="liblinear",
        max_iter=2000,
    )
    final_clf.fit(X_values, y_values)

    return final_clf, auc, oof

def run_public_clone_l1_model(df, min_pos_repertoires=3, min_log_odds=1.0, C=1.0):
    all_stats, enriched = find_enriched_public_clones(
        df,
        min_pos_repertoires=min_pos_repertoires,
        min_log_odds=min_log_odds,
        use_vj=True,
    )

    selected_clone_ids = enriched["clone_id"]

    X_pub = build_public_clone_presence_matrix(
        df,
        selected_clone_ids,
        use_vj=True,
    )

    y = (
        df[["ID", "label_positive"]]
        .drop_duplicates()
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_pub.index]
    )

    model, auc, oof = fit_l1_logit_cv(X_pub, y, C=C, n_splits=5, random_state=0)
    return model, auc, oof, X_pub, y, all_stats, enriched
# Function to perform the full feature selection and training process within CV
def run_public_clone_l1_model_no_leak(df, min_pos_repertoires=3, min_log_odds=1.0, C=1.0, n_splits=5, random_state=0):
    
    # 1. Get repertoire labels (y) and IDs (X index) once
    rep_label = df[["ID", "label_positive"]].drop_duplicates("ID").set_index("ID")
    y = rep_label["label_positive"].astype(int)
    
    # 2. Get unique repertoire IDs to use for CV split
    rep_ids = rep_label.index.to_numpy() 
    y_values = y.to_numpy() 

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof = np.zeros(len(y_values), dtype=float)

    # 3. Iterate over the folds
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(rep_ids, y_values)):
        # Get repertoire IDs for training and validation sets
        train_rep_ids = rep_ids[train_idx]
        val_rep_ids = rep_ids[val_idx]

        # --- A. FEATURE SELECTION ON TRAINING DATA ONLY ---
        df_train = df[df["ID"].isin(train_rep_ids)].copy() 
        df_val = df[df["ID"].isin(val_rep_ids)].copy()
        
        # Select enriched clones based *ONLY* on the training data
        _, enriched_train = find_enriched_public_clones(
            df_train,
            min_pos_repertoires=min_pos_repertoires,
            min_log_odds=min_log_odds,
            use_vj=True,
        )
        selected_clone_ids = enriched_train["clone_id"]

        # --- B. FEATURE TRANSFORMATION ON TRAIN/VAL DATA ---
        # Build feature matrix for TRAIN and VAL using the features selected on TRAIN
        X_train = build_public_clone_presence_matrix(df_train, selected_clone_ids, use_vj=True)
        X_val = build_public_clone_presence_matrix(df_val, selected_clone_ids, use_vj=True)

        # Ensure y_train and y_val align with X_train and X_val indices
        y_train = y.loc[X_train.index]
        y_val = y.loc[X_val.index]

        # --- C. MODEL FITTING AND PREDICTION ---
        clf = LogisticRegression(
            penalty="l1",
            C=C,
            solver="liblinear",
            max_iter=10000,
        )
        clf.fit(X_train.values, y_train.values)
        
        # Store predictions for OOF calculation
        # The indices (val_idx) are based on the original rep_ids/y_values array order
        val_map = {id_: idx for idx, id_ in zip(val_idx, y_val.index)}
        original_val_indices = np.array([val_map[id_] for id_ in X_val.index])
        oof[original_val_indices] = clf.predict_proba(X_val.values)[:, 1]

    # Final OOF AUC
    auc = roc_auc_score(y_values, oof)
    
    # You would still need to train the final model on all data *after* selecting 
    # features on all data, but the OOF AUC is now unbiased.
    # The final model training (on ALL data) remains the same as your original run_public_clone_l1_model, 
    # but for an unbiased performance estimate, the OOF AUC is the key.
    # The original logic for final model training can be put here if needed.
    
    return auc, oof

In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed


def find_enriched_public_clones(
    df,
    min_pos_repertoires=3,
    min_log_odds=1.0,
    use_vj=True,
):
    df = df.copy()
    if use_vj:
        df["clone_id"] = (
            df["junction_aa"].astype("string")
            + "|" + df["v_call"].astype("string")
            + "|" + df["j_call"].astype("string")
        )
    else:
        df["clone_id"] = df["junction_aa"].astype("string")

    rep_label = df[["ID", "label_positive"]].drop_duplicates("ID")
    rep_label["label_positive"] = rep_label["label_positive"].astype(np.int8)

    n_pos = int((rep_label["label_positive"] == 1).sum())
    n_neg = int((rep_label["label_positive"] == 0).sum())

    tmp = df[["clone_id", "ID", "label_positive"]].drop_duplicates(["clone_id", "ID"])

    ct = pd.crosstab(tmp["clone_id"], tmp["label_positive"])
    ct = ct.reindex(columns=[0, 1], fill_value=0).astype(np.int32)
    ct = ct.rename(columns={0: "neg_reps", 1: "pos_reps"})

    alpha = 0.5
    pos = ct["pos_reps"].to_numpy()
    neg = ct["neg_reps"].to_numpy()

    log_odds = np.log(
        (pos + alpha) / (n_pos - pos + alpha)
    ) - np.log(
        (neg + alpha) / (n_neg - neg + alpha)
    )

    ct = ct.assign(log_odds=log_odds, clone_id=ct.index).reset_index(drop=True)

    enriched = ct[
        (ct["pos_reps"] >= min_pos_repertoires) &
        (ct["log_odds"] >= min_log_odds)
    ].reset_index(drop=True)

    return ct, enriched


def build_public_clone_presence_matrix(df, selected_clone_ids, use_vj=True):
    df = df.copy()
    if use_vj:
        df["clone_id"] = (
            df["junction_aa"].astype("string")
            + "|" + df["v_call"].astype("string")
            + "|" + df["j_call"].astype("string")
        )
    else:
        df["clone_id"] = df["junction_aa"].astype("string")

    df_sel = df[df["clone_id"].isin(selected_clone_ids)].copy()
    df_sel = df_sel[["ID", "clone_id"]].drop_duplicates()
    df_sel["value"] = 1.0

    X = (
        df_sel
        .pivot(index="ID", columns="clone_id", values="value")
        .reindex(columns=selected_clone_ids)
        .fillna(0.0)
        .astype(np.float32)
    )
    return X


def _run_fold(
    fold_id,
    train_ids,
    val_ids,
    df,
    C,
    min_pos_repertoires,
    min_log_odds,
    use_vj,
):
    print(f"[fold {fold_id+1}] start: n_train_ids={len(train_ids)}, n_val_ids={len(val_ids)}")

    df_train = df[df["ID"].isin(train_ids)]
    df_val = df[df["ID"].isin(val_ids)]

    all_stats_tr, enriched_tr = find_enriched_public_clones(
        df_train,
        min_pos_repertoires=min_pos_repertoires,
        min_log_odds=min_log_odds,
        use_vj=use_vj,
    )
    selected = enriched_tr["clone_id"]
    n_sel = len(selected)

    if n_sel == 0:
        rep_label_val = (
            df_val[["ID", "label_positive"]]
            .drop_duplicates("ID")
            .set_index("ID")["label_positive"]
            .astype(int)
        )
        base_rate = rep_label_val.mean()
        val_pred = np.full(len(rep_label_val), base_rate, dtype=float)
        fold_auc = roc_auc_score(rep_label_val.values, val_pred)
        print(f"[fold {fold_id+1}] auc={fold_auc:.4f}, n_selected={n_sel} (constant preds)")
        return fold_id, rep_label_val.index.to_numpy(), val_pred, fold_auc, n_sel

    X_train = build_public_clone_presence_matrix(df_train, selected, use_vj=use_vj)
    X_val = build_public_clone_presence_matrix(df_val, selected, use_vj=use_vj)

    y_train = (
        df_train[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_train.index]
    )
    y_val = (
        df_val[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_val.index]
    )

    clf = LogisticRegression(
        penalty="l1",
        C=C,
        solver="liblinear",
        max_iter=10000,
    )
    clf.fit(X_train.values, y_train.values)
    val_pred = clf.predict_proba(X_val.values)[:, 1]
    fold_auc = roc_auc_score(y_val.values, val_pred)

    print(f"[fold {fold_id+1}] auc={fold_auc:.4f}, n_selected={n_sel}")
    return fold_id, y_val.index.to_numpy(), val_pred, fold_auc, n_sel




In [20]:
def predict_public_clone_model(df_test, final_clf, enriched_public, use_vj=True):
    df_test = df_test.copy()
    rep_ids = df_test["ID"].drop_duplicates().to_numpy()

    if enriched_public is None or len(enriched_public) == 0:
        X_test = pd.DataFrame(
            {"bias": np.ones(len(rep_ids), dtype=np.float32)},
            index=rep_ids,
        )
    else:
        selected_clone_ids = enriched_public["clone_id"]
        X_test = build_public_clone_presence_matrix(
            df_test,
            selected_clone_ids=selected_clone_ids,
            use_vj=use_vj,
        )
        missing_ids = [rid for rid in rep_ids if rid not in X_test.index]
        if len(missing_ids) > 0:
            extra = pd.DataFrame(
                0.0,
                index=missing_ids,
                columns=X_test.columns,
                dtype=np.float32,
            )
            X_test = (
                pd.concat([X_test, extra], axis=0)
                .loc[rep_ids]
            )

    test_pred = final_clf.predict_proba(X_test.values)[:, 1]
    pred_df = pd.DataFrame(
        {"ID": X_test.index.to_numpy(), "pred_public": test_pred}
    )
    return pred_df, X_test
def stats_to_seq_table(stats_df, dataset_name, start_rank=1):
    df = stats_df.copy().reset_index(drop=True)

    # split clone_id -> junction_aa, v_call, j_call
    split = df["clone_id"].astype(str).str.split("|", n=2, expand=True)
    df["junction_aa"] = split[0]
    df["v_call"] = split[1]
    df["j_call"] = split[2]

    # rank for constructing ID
    ranks = np.arange(start_rank, start_rank + len(df))

    out = pd.DataFrame(
        {
            "ID": [f"{dataset_name}_seq_top_{r}" for r in ranks],
            "dataset": dataset_name,
            "label_positive_probability": -999.0,
            "junction_aa": df["junction_aa"].values,
            "v_call": df["v_call"].values,
            "j_call": df["j_call"].values,
        }
    )

    return out
def preds_to_submission(pred_df, dataset_name):
    out = pd.DataFrame(
        {
            "ID": pred_df["ID"].values,
            "dataset": dataset_name,
            "label_positive_probability": pred_df["pred_public"].astype(float).values,
            "junction_aa": -999.0,
            "v_call": -999.0,
            "j_call": -999.0,
        }
    )
    return out
from joblib import Parallel, delayed
import numpy as np

def eval_param_combo(df, min_pos, min_lo, C, n_splits=5, random_state=0):
    print(f"=== combo: min_pos={min_pos}, min_log_odds={min_lo}, C={C} ===")
    final_clf, cv_auc, oof, X_pub, y_pub, all_stats, enriched_public, fold_aucs = run_public_clone_l1_cv_noleak(
        df,
        min_pos_repertoires=min_pos,
        min_log_odds=min_lo,
        C=C,
        n_splits=n_splits,
        n_jobs=1,
        use_vj=True,
    )
    print(f"=== combo done: min_pos={min_pos}, min_log_odds={min_lo}, C={C}, cv_auc={cv_auc:.4f} ===")
    return {
        "min_pos": min_pos,
        "min_log_odds": min_lo,
        "C": C,
        "cv_auc": cv_auc,
        "fold_aucs": fold_aucs,
    }

In [21]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


def _make_logreg(penalty, C, class_weight, l1_ratio):
    if penalty == "elasticnet":
        if l1_ratio is None:
            l1_ratio = 0.5
        return LogisticRegression(
            penalty="elasticnet",
            C=C,
            solver="saga",
            max_iter=10000,
            class_weight=class_weight,
            l1_ratio=l1_ratio,
        )
    elif penalty in ("l1", "l2"):
        return LogisticRegression(
            penalty=penalty,
            C=C,
            solver="liblinear",
            max_iter=10000,
            class_weight=class_weight,
        )
    else:
        raise ValueError(f"unsupported penalty: {penalty}")


def _run_fold(
    fold_id,
    train_ids,
    val_ids,
    df,
    C,
    min_pos_repertoires,
    min_log_odds,
    use_vj,
    penalty_cv="l1",
    class_weight=None,
    l1_ratio_cv=None,
    extra_clone_ids=None,
):
    print(f"[fold {fold_id+1}] start: n_train_ids={len(train_ids)}, n_val_ids={len(val_ids)}")

    df_train = df[df["ID"].isin(train_ids)]
    df_val = df[df["ID"].isin(val_ids)]

    all_stats_tr, enriched_tr = find_enriched_public_clones(
        df_train,
        min_pos_repertoires=min_pos_repertoires,
        min_log_odds=min_log_odds,
        use_vj=use_vj,
        extra_clone_ids=extra_clone_ids,
    )
    selected = enriched_tr["clone_id"]
    n_sel = len(selected)

    if n_sel == 0:
        rep_label_val = (
            df_val[["ID", "label_positive"]]
            .drop_duplicates("ID")
            .set_index("ID")["label_positive"]
            .astype(int)
        )
        base_rate = rep_label_val.mean()
        val_pred = np.full(len(rep_label_val), base_rate, dtype=float)
        fold_auc = roc_auc_score(rep_label_val.values, val_pred)
        print(f"[fold {fold_id+1}] auc={fold_auc:.4f}, n_selected={n_sel} (constant preds)")
        return fold_id, rep_label_val.index.to_numpy(), val_pred, fold_auc, n_sel

    X_train = build_public_clone_presence_matrix(df_train, selected, use_vj=use_vj)
    X_val = build_public_clone_presence_matrix(df_val, selected, use_vj=use_vj)

    y_train = (
        df_train[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_train.index]
    )
    y_val = (
        df_val[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_val.index]
    )

    clf = _make_logreg(
        penalty=penalty_cv,
        C=C,
        class_weight=class_weight,
        l1_ratio=l1_ratio_cv,
    )
    clf.fit(X_train.values, y_train.values)
    val_pred = clf.predict_proba(X_val.values)[:, 1]
    fold_auc = roc_auc_score(y_val.values, val_pred)

    print(f"[fold {fold_id+1}] auc={fold_auc:.4f}, n_selected={n_sel}")
    return fold_id, y_val.index.to_numpy(), val_pred, fold_auc, n_sel


def run_public_clone_l1_cv_noleak(
    df,
    min_pos_repertoires=3,
    min_log_odds=1.0,
    C=1.0,
    n_splits=5,
    random_state=0,
    n_jobs=-1,
    use_vj=True,
    penalty_cv="l1",
    penalty_final=None,
    class_weight=None,
    l1_ratio_cv=None,
    l1_ratio_final=None,
    extra_clone_ids=None,
):
    rep_label = df[["ID", "label_positive"]].drop_duplicates("ID")
    rep_label["label_positive"] = rep_label["label_positive"].astype(int)
    ids = rep_label["ID"].to_numpy()
    y = rep_label["label_positive"].to_numpy().astype(int)

    counts = np.bincount(y, minlength=2)
    if (counts > 0).sum() < 2:
        return None, -1.0, None, None, None, None, None, []
    
    n_splits = min(n_splits, int(counts.min()))
    if n_splits < 2:
        return None, -1.0, None, None, None, None, None, []
    id_to_idx = {rid: i for i, rid in enumerate(ids)}

    skf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=random_state,
    )

    fold_specs = []
    for fold_id, (train_idx, val_idx) in enumerate(skf.split(ids, y)):
        fold_specs.append((fold_id, ids[train_idx], ids[val_idx]))

    results = Parallel(n_jobs=n_jobs)(
        delayed(_run_fold)(
            fold_id,
            train_ids,
            val_ids,
            df,
            C,
            min_pos_repertoires,
            min_log_odds,
            use_vj,
            penalty_cv,
            class_weight,
            l1_ratio_cv,
            extra_clone_ids,
        )
        for fold_id, train_ids, val_ids in fold_specs
    )

    oof = np.zeros(len(ids), dtype=float)
    fold_aucs = []
    for fold_id, val_ids, val_pred, fold_auc, n_sel in results:
        idxs = [id_to_idx[rid] for rid in val_ids]
        oof[idxs] = val_pred
        fold_aucs.append(fold_auc)

    cv_auc = roc_auc_score(y, oof)
    print(f"[cv] overall auc={cv_auc:.4f}")

    all_stats_full, enriched_full = find_enriched_public_clones(
        df,
        min_pos_repertoires=min_pos_repertoires,
        min_log_odds=min_log_odds,
        use_vj=use_vj,
        extra_clone_ids=extra_clone_ids,
    )
    selected_full = enriched_full["clone_id"]

    if len(selected_full) == 0:
        print("[full] no clones selected; training intercept-only model")
        X_full = pd.DataFrame(
            {"bias": np.ones(len(rep_label), dtype=np.float32)},
            index=rep_label["ID"].values,
        )
    else:
        print(f"[full] n_selected={len(selected_full)}")
        X_full = build_public_clone_presence_matrix(df, selected_full, use_vj=use_vj)

    y_full = (
        df[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_full.index]
    )

    if penalty_final is None:
        penalty_final = penalty_cv
    if l1_ratio_final is None:
        l1_ratio_final = l1_ratio_cv

    final_clf = _make_logreg(
        penalty=penalty_final,
        C=C,
        class_weight=class_weight,
        l1_ratio=l1_ratio_final,
    )
    final_clf.fit(X_full.values, y_full.values)

    return final_clf, cv_auc, oof, X_full, y_full, all_stats_full, enriched_full, fold_aucs

In [22]:
import numpy as np
import pandas as pd

def find_enriched_public_clones(
    df,
    min_pos_repertoires=3,
    min_log_odds=1.0,
    use_vj=True,
    extra_clone_ids=None,
):
    df = df.copy()
    if use_vj:
        df["clone_id"] = (
            df["junction_aa"].astype("string")
            + "|" + df["v_call"].astype("string")
            + "|" + df["j_call"].astype("string")
        )
    else:
        df["clone_id"] = df["junction_aa"].astype("string")

    rep_label = df[["ID", "label_positive"]].drop_duplicates("ID")
    rep_label["label_positive"] = rep_label["label_positive"].astype(np.int8)

    n_pos = int((rep_label["label_positive"] == 1).sum())
    n_neg = int((rep_label["label_positive"] == 0).sum())

    tmp = df[["clone_id", "ID", "label_positive"]].drop_duplicates(["clone_id", "ID"])

    ct = pd.crosstab(tmp["clone_id"], tmp["label_positive"])
    ct = ct.reindex(columns=[0, 1], fill_value=0).astype(np.int32)
    ct = ct.rename(columns={0: "neg_reps", 1: "pos_reps"})

    alpha = 0.5
    pos = ct["pos_reps"].to_numpy()
    neg = ct["neg_reps"].to_numpy()

    log_odds = np.log(
        (pos + alpha) / (n_pos - pos + alpha)
    ) - np.log(
        (neg + alpha) / (n_neg - neg + alpha)
    )

    ct = ct.assign(log_odds=log_odds, clone_id=ct.index).reset_index(drop=True)

    enriched = ct[
        (ct["pos_reps"] >= min_pos_repertoires) &
        (ct["log_odds"] >= min_log_odds)
    ].reset_index(drop=True)

    if extra_clone_ids is not None:
        extra_clone_ids = set(extra_clone_ids)
        extra_mask = ct["clone_id"].isin(extra_clone_ids)
        extra_df = ct[extra_mask & ~ct["clone_id"].isin(enriched["clone_id"])]
        if not extra_df.empty:
            enriched = pd.concat([enriched, extra_df], ignore_index=True)

    return ct, enriched

In [23]:
def _run_fold(
    fold_id,
    train_ids,
    val_ids,
    df,
    C,
    min_pos_repertoires,
    min_log_odds,
    use_vj,
    penalty_cv="l1",
    class_weight=None,
    l1_ratio_cv=None,
    extra_clone_ids=None,
):
    print(f"[fold {fold_id+1}] start: n_train_ids={len(train_ids)}, n_val_ids={len(val_ids)}")

    df_train = df[df["ID"].isin(train_ids)]
    df_val = df[df["ID"].isin(val_ids)]

    # --- 1. Feature Selection ---
    all_stats_tr, enriched_tr = find_enriched_public_clones(
        df_train,
        min_pos_repertoires=min_pos_repertoires,
        min_log_odds=min_log_odds,
        use_vj=use_vj,
        extra_clone_ids=extra_clone_ids,
    )
    selected = enriched_tr["clone_id"]
    n_sel = len(selected)

    # Handle case where NO clones were found
    if n_sel == 0:
        rep_label_val = (
            df_val[["ID", "label_positive"]]
            .drop_duplicates("ID")
            .set_index("ID")["label_positive"]
            .astype(int)
        )
        base_rate = rep_label_val.mean()
        val_pred = np.full(len(rep_label_val), base_rate, dtype=float)
        try:
            fold_auc = roc_auc_score(rep_label_val.values, val_pred)
        except ValueError:
            fold_auc = np.nan
        print(f"[fold {fold_id+1}] auc={fold_auc}, n_selected={n_sel} (constant preds, no clones)")
        return fold_id, rep_label_val.index.to_numpy(), val_pred, fold_auc, n_sel

    # --- 2. Build Matrices ---
    X_train = build_public_clone_presence_matrix(df_train, selected, use_vj=use_vj)
    X_val = build_public_clone_presence_matrix(df_val, selected, use_vj=use_vj)

    # --- 3. CRITICAL FIX: Reindex to include subjects with 0 counts ---
    # This brings back the Negative subjects who were dropped because 
    # they didn't have any of the "enriched" clones.
    X_train = X_train.reindex(index=train_ids, fill_value=0)
    X_val = X_val.reindex(index=val_ids, fill_value=0)

    # --- 4. Align Targets ---
    # Now y_train will correspond exactly to train_ids (including Negatives)
    y_train = (
        df[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_train.index]
    )
    
    y_val = (
        df[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_val.index]
    )

    # --- 5. Safety Checks ---
    if y_train.nunique() < 2:
        # This should theoretically not happen now, unless the fold SPLIT itself 
        # put only Positives in train (which StratifiedKFold prevents).
        base_rate = y_train.mean()
        val_pred = np.full(len(y_val), base_rate, dtype=float)
        try:
            fold_auc = roc_auc_score(y_val.values, val_pred)
        except ValueError:
            fold_auc = np.nan
        print(
            f"[fold {fold_id+1}] single-class y_train (after reindex), using constant preds; "
            f"auc={fold_auc}, n_selected={n_sel}"
        )
        return fold_id, y_val.index.to_numpy(), val_pred, fold_auc, n_sel

    # --- 6. Train & Predict ---
    clf = _make_logreg(
        penalty=penalty_cv,
        C=C,
        class_weight=class_weight,
        l1_ratio=l1_ratio_cv,
    )
    
    clf.fit(X_train.values, y_train.values)
    val_pred = clf.predict_proba(X_val.values)[:, 1]
    
    try:
        fold_auc = roc_auc_score(y_val.values, val_pred)
    except ValueError:
        fold_auc = np.nan

    print(f"[fold {fold_id+1}] auc={fold_auc:.4f}, n_selected={n_sel}")
    return fold_id, y_val.index.to_numpy(), val_pred, fold_auc, n_sel

In [24]:
import os
import pandas as pd
import numpy as np
import random
from joblib import Parallel, delayed
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# ---------------------------------------------------------
# 1. DEFINE FUNCTIONS (All in one place to ensure freshness)
# ---------------------------------------------------------

def _make_logreg(penalty, C, class_weight, l1_ratio):
    if penalty == "elasticnet":
        if l1_ratio is None:
            l1_ratio = 0.5
        return LogisticRegression(
            penalty="elasticnet",
            C=C,
            solver="saga",
            max_iter=10000,
            class_weight=class_weight,
            l1_ratio=l1_ratio,
        )
    elif penalty in ("l1", "l2"):
        return LogisticRegression(
            penalty=penalty,
            C=C,
            solver="liblinear",
            max_iter=10000,
            class_weight=class_weight,
        )
    else:
        raise ValueError(f"unsupported penalty: {penalty}")

def _run_fold(
    fold_id,
    train_ids,
    val_ids,
    df,
    C,
    min_pos_repertoires,
    min_log_odds,
    use_vj,
    penalty_cv,
    class_weight,
    l1_ratio_cv,
    extra_clone_ids
):
    # Filter data
    df_train = df[df["ID"].isin(train_ids)]
    df_val = df[df["ID"].isin(val_ids)]

    # --- Feature Selection ---
    all_stats_tr, enriched_tr = find_enriched_public_clones(
        df_train,
        min_pos_repertoires=min_pos_repertoires,
        min_log_odds=min_log_odds,
        use_vj=use_vj,
        extra_clone_ids=extra_clone_ids,
    )
    selected = enriched_tr["clone_id"]
    n_sel = len(selected)

    # --- Edge Case: No Features ---
    if n_sel == 0:
        rep_label_val = (
            df_val[["ID", "label_positive"]]
            .drop_duplicates("ID")
            .set_index("ID")["label_positive"]
            .astype(int)
        )
        base_rate = rep_label_val.mean()
        val_pred = np.full(len(rep_label_val), base_rate, dtype=float)
        try:
            fold_auc = roc_auc_score(rep_label_val.values, val_pred)
        except ValueError:
            fold_auc = 0.5 
        return fold_id, rep_label_val.index.to_numpy(), val_pred, fold_auc, n_sel

    # --- Build Matrices ---
    X_train = build_public_clone_presence_matrix(df_train, selected, use_vj=use_vj)
    X_val = build_public_clone_presence_matrix(df_val, selected, use_vj=use_vj)

    # --- CRITICAL FIX: Reindex to force inclusion of subjects with 0 clones ---
    # This brings back the Negative subjects who were dropped because they have no clones.
    # We fill their rows with 0.0.
    X_train = X_train.reindex(index=train_ids, fill_value=0.0)
    X_val = X_val.reindex(index=val_ids, fill_value=0.0)

    # --- Align Targets ---
    y_train = (
        df[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_train.index]
    )
    
    y_val = (
        df[["ID", "label_positive"]]
        .drop_duplicates("ID")
        .set_index("ID")["label_positive"]
        .astype(int)
        .loc[X_val.index]
    )

    # --- Safety Check: Ensure 2 classes ---
    if y_train.nunique() < 2:
        # If we still have only 1 class, return constant prediction (0.5 AUC)
        return fold_id, y_val.index.to_numpy(), np.zeros(len(y_val)), 0.5, n_sel

    # --- Train & Predict ---
    clf = _make_logreg(
        penalty=penalty_cv,
        C=C,
        class_weight=class_weight,
        l1_ratio=l1_ratio_cv,
    )
    
    clf.fit(X_train.values, y_train.values)
    val_pred = clf.predict_proba(X_val.values)[:, 1]
    
    try:
        fold_auc = roc_auc_score(y_val.values, val_pred)
    except ValueError:
        fold_auc = 0.5

    return fold_id, y_val.index.to_numpy(), val_pred, fold_auc, n_sel

def run_public_clone_l1_cv_noleak(
    df,
    min_pos_repertoires=3,
    min_log_odds=1.0,
    C=1.0,
    n_splits=5,
    random_state=0,
    n_jobs=2,  # Default to 1 for inner loop
    use_vj=True,
    penalty_cv="l1",
    penalty_final=None,
    class_weight=None,
    l1_ratio_cv=None,
    l1_ratio_final=None,
    extra_clone_ids=None,
):
    rep_label = df[["ID", "label_positive"]].drop_duplicates("ID")
    rep_label["label_positive"] = rep_label["label_positive"].astype(int)
    ids = rep_label["ID"].to_numpy()
    y = rep_label["label_positive"].to_numpy().astype(int)

    # Basic check for enough data
    counts = np.bincount(y, minlength=2)
    if (counts > 0).sum() < 2:
        return None, -1.0, None, None, None, None, None, []
    
    n_splits = min(n_splits, int(counts.min()))
    if n_splits < 2:
        return None, -1.0, None, None, None, None, None, []
    id_to_idx = {rid: i for i, rid in enumerate(ids)}

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    fold_specs = []
    for fold_id, (train_idx, val_idx) in enumerate(skf.split(ids, y)):
        fold_specs.append((fold_id, ids[train_idx], ids[val_idx]))

    # Run Folds
    results = Parallel(n_jobs=n_jobs)(
        delayed(_run_fold)(
            fold_id, train_ids, val_ids, df, C, min_pos_repertoires, min_log_odds, use_vj,
            penalty_cv, class_weight, l1_ratio_cv, extra_clone_ids
        )
        for fold_id, train_ids, val_ids in fold_specs
    )

    oof = np.zeros(len(ids), dtype=float)
    fold_aucs = []
    for fold_id, val_ids, val_pred, fold_auc, n_sel in results:
        idxs = [id_to_idx[rid] for rid in val_ids]
        oof[idxs] = val_pred
        fold_aucs.append(fold_auc)

    cv_auc = roc_auc_score(y, oof)

    # Final Model Training
    all_stats_full, enriched_full = find_enriched_public_clones(
        df, min_pos_repertoires=min_pos_repertoires, min_log_odds=min_log_odds, use_vj=use_vj, extra_clone_ids=extra_clone_ids
    )
    selected_full = enriched_full["clone_id"]

    if len(selected_full) == 0:
        X_full = pd.DataFrame({"bias": np.ones(len(rep_label), dtype=np.float32)}, index=rep_label["ID"].values)
    else:
        X_full = build_public_clone_presence_matrix(df, selected_full, use_vj=use_vj)
        # CRITICAL: Reindex final model matrix too
        X_full = X_full.reindex(index=ids, fill_value=0.0)

    y_full = df[["ID", "label_positive"]].drop_duplicates("ID").set_index("ID")["label_positive"].astype(int).loc[X_full.index]

    if penalty_final is None: penalty_final = penalty_cv
    if l1_ratio_final is None: l1_ratio_final = l1_ratio_cv

    final_clf = _make_logreg(penalty=penalty_final, C=C, class_weight=class_weight, l1_ratio=l1_ratio_final)
    final_clf.fit(X_full.values, y_full.values)

    return final_clf, cv_auc, oof, X_full, y_full, all_stats_full, enriched_full, fold_aucs

# ---------------------------------------------------------
# 2. RUN SEARCH
# ---------------------------------------------------------

log_file = "training_auc_05E_log.tsv"
log_file = "training_auc_05E_log.tsv"


pairs = [
    ('train_datasets/train_datasets/train_dataset_1',
     ['test_datasets/test_datasets/test_dataset_1'])
]


penalty_options     = ["l1", "elasticnet"]


C_values = [10.0, 50.0, 80.0, 100.0]


min_pos_values = [2]


min_log_odds_values = [2.0,3.0, 4.0, 5.0]

# High C values are excellent here. 
# They force the model to pay attention to the few rare signals it finds.
C_values = [10.0, 50.0, 80.0, 100.0]

# Keep strictly closer to 1.0 (Lasso) to zero out noise.
l1_ratio_options = [0.95, 0.975, 1.0] 

# Keep balanced
class_weight_options = ["balanced"]
#tried N_TRIALS=100, 200,250,300

N_TRIALS = 400
N_SPLITS = 5
RANDOM_STATE = 123

N_FOLD_JOBS = 1        
PARALLEL_JOBS = 35    


submission_out_dir = "results_05E_Pub_Clones"
important_out_dir = "results_05E_Pub_Clones"
os.makedirs(submission_out_dir, exist_ok=True)
os.makedirs(important_out_dir, exist_ok=True)

for train_dir, test_dirs in pairs:
    train_name = os.path.basename(train_dir)
    imp_out_path = os.path.join(important_out_dir, f"{train_name}_important_sequences.tsv")
    submission_paths = [os.path.join(submission_out_dir, f"{os.path.basename(td)}_submission.tsv") for td in test_dirs]

    if os.path.exists(imp_out_path) and all(os.path.exists(p) for p in submission_paths):
        print(f"Skipping {train_name}: all outputs already exist.")
        continue

    full_df = load_full_dataset(train_dir)

    def run_random_trial(trial_idx):
        min_pos = random.choice(min_pos_values)
        min_lo = random.choice(min_log_odds_values)
        C = random.choice(C_values)
        penalty = random.choice(penalty_options)
        class_weight = random.choice(class_weight_options)
        l1_ratio = random.choice(l1_ratio_options) if penalty == "elasticnet" else None
        
        try:
            # We call the CV function with n_jobs=1 because we are already parallelized here
            _, cv_auc, _, _, _, _, _, _ = run_public_clone_l1_cv_noleak(
                full_df,
                min_pos_repertoires=min_pos,
                min_log_odds=min_lo,
                C=C,
                n_splits=N_SPLITS,
                random_state=RANDOM_STATE,
                n_jobs=1,        # FIXED: Inner loop sequential
                use_vj=True,     # FIXED: Explicit True
                penalty_cv=penalty,
                penalty_final=penalty,
                class_weight=class_weight,
                l1_ratio_cv=l1_ratio,
                l1_ratio_final=l1_ratio,
            )
        except Exception as e:
            # Catch errors so one bad trial doesn't kill the batch
            # print(f"Trial {trial_idx} failed: {e}") 
            cv_auc = -1.0
    
        return {
            "trial_idx": trial_idx, "cv_auc": cv_auc,
            "params": {
                "min_pos_repertoires": min_pos, "min_log_odds": min_lo, "C": C,
                "penalty": penalty, "class_weight": class_weight, "l1_ratio": l1_ratio
            }
        }

    print(f"Starting Joblib Random Search for {train_name} using {PARALLEL_JOBS} cores...")
    
    results = Parallel(n_jobs=PARALLEL_JOBS, verbose=1)(
        delayed(run_random_trial)(i) for i in range(N_TRIALS)
    )

    valid_results = [r for r in results if r["cv_auc"] >= 0]
    if not valid_results:
        print(f"All trials failed for {train_name}.")
        continue

    best_result = max(valid_results, key=lambda x: x["cv_auc"])
    best_params = best_result["params"]
    print(f"{train_name} Best AUC: {best_result['cv_auc']:.4f}")
    print(f"Best Params: {best_params}")

    # Retrain final model (can use parallel folds here since outer loop is done)
    final_clf, cv_auc, oof_public, X_pub, y_pub, all_stats, enriched_public, fold_aucs = run_public_clone_l1_cv_noleak(
        full_df,
        min_pos_repertoires=best_params["min_pos_repertoires"],
        min_log_odds=best_params["min_log_odds"],
        C=best_params["C"],
        n_splits=N_SPLITS,
        random_state=RANDOM_STATE,
        n_jobs=PARALLEL_JOBS, # Now we can use parallel cores for the final fit
        use_vj=True,
        penalty_cv=best_params["penalty"],
        penalty_final=best_params["penalty"],
        class_weight=best_params["class_weight"],
        l1_ratio_cv=best_params["l1_ratio"],
        l1_ratio_final=best_params["l1_ratio"],
    )

    with open(log_file, "a") as f:
        f.write(f"{train_name}\t{best_params['min_pos_repertoires']}\t{best_params['min_log_odds']}\t{best_params['C']}\t{best_params['penalty']}\t{best_params['class_weight']}\t{best_params['l1_ratio']}\t{cv_auc}\n")

    # Generate Predictions and Important Sequences
    for test_dir in test_dirs:
        test_name = os.path.basename(test_dir)
        out_path = os.path.join(submission_out_dir, f"{test_name}_submission.tsv")
        if os.path.exists(out_path): continue

        df_test = load_full_dataset(test_dir)
        pred_test_public, X_test_public = predict_public_clone_model(df_test, final_clf=final_clf, enriched_public=enriched_public, use_vj=True)

        if "label_positive" in df_test.columns:
            y_test = df_test[["ID", "label_positive"]].drop_duplicates("ID").set_index("ID")["label_positive"].astype(int).loc[pred_test_public["ID"]].to_numpy()
            try:
                test_auc = roc_auc_score(y_test, pred_test_public["pred_public"])
            except: test_auc = 0.5
            print(f"{test_name} test auc: {test_auc:.4f}")
            with open(log_file, "a") as f: f.write(f"{train_name}\t{test_name}\ttest_auc_public\t{test_auc}\n")

        preds_to_submission(pred_test_public, dataset_name=test_name).to_csv(out_path, sep="\t", index=False)
        print(f"wrote {out_path}")

    if not os.path.exists(imp_out_path):
        n_top = 50000
        all_stats_sorted = all_stats.sort_values("log_odds", ascending=False)
        enriched_ids = set(enriched_public["clone_id"])
        extra_stats = all_stats_sorted[~all_stats_sorted["clone_id"].isin(enriched_ids)].head(max(0, n_top - len(enriched_ids)))
        top_stats = pd.concat([enriched_public, extra_stats], ignore_index=True).drop_duplicates("clone_id")
        stats_to_seq_table(top_stats, dataset_name=train_name, start_rank=1).to_csv(imp_out_path, sep="\t", index=False)
        print(f"wrote {imp_out_path}")

Loading files: 100%|██████████| 400/400 [00:05<00:00, 75.18it/s]


Starting Joblib Random Search for train_dataset_1 using 35 cores...


[Parallel(n_jobs=35)]: Using backend LokyBackend with 35 concurrent workers.
[Parallel(n_jobs=35)]: Done 130 tasks      | elapsed: 66.3min
[Parallel(n_jobs=35)]: Done 400 out of 400 | elapsed: 186.7min finished


train_dataset_1 Best AUC: 0.5306
Best Params: {'min_pos_repertoires': 2, 'min_log_odds': 2.0, 'C': 10.0, 'penalty': 'l1', 'class_weight': 'balanced', 'l1_ratio': None}


Loading files: 100%|██████████| 400/400 [00:04<00:00, 81.77it/s]


wrote results_05E_Pub_Clones/test_dataset_1_submission.tsv
wrote results_05E_Pub_Clones/train_dataset_1_important_sequences.tsv


In [25]:
import os
import pandas as pd
import random
from joblib import Parallel, delayed  
from sklearn.metrics import roc_auc_score

log_file = "training_auc_05E_log.tsv"


pairs = [
    ('train_datasets/train_datasets/train_dataset_3',
     ['test_datasets/test_datasets/test_dataset_3'])
]


min_pos_values      = [ 6]
min_log_odds_values = [1.6, 1.65, 1.7,1.75]
C_values            = [4.0,4.5, 5.0,5.5,6.0]
penalty_options     = [ "elasticnet"]
class_weight_options = [ None, "balanced"]
l1_ratio_options    = [0.75, 0.9, 0.8, 0.85]

#tried N_TRIALS=100, 200,250,300

N_TRIALS = 400
N_SPLITS = 5
RANDOM_STATE = 123

N_FOLD_JOBS = 1        
PARALLEL_JOBS = 35    


submission_out_dir = "results_05E_Pub_Clones"
important_out_dir = "results_05E_Pub_Clones"
os.makedirs(submission_out_dir, exist_ok=True)
os.makedirs(important_out_dir, exist_ok=True)

for train_dir, test_dirs in pairs:
    train_name = os.path.basename(train_dir)
    imp_out_path = os.path.join(important_out_dir, f"{train_name}_important_sequences.tsv")
    submission_paths = [os.path.join(submission_out_dir, f"{os.path.basename(td)}_submission.tsv") for td in test_dirs]

    if os.path.exists(imp_out_path) and all(os.path.exists(p) for p in submission_paths):
        print(f"Skipping {train_name}: all outputs already exist.")
        continue

    full_df = load_full_dataset(train_dir)

    def run_random_trial(trial_idx):
        min_pos = random.choice(min_pos_values)
        min_lo = random.choice(min_log_odds_values)
        C = random.choice(C_values)
        penalty = random.choice(penalty_options)
        class_weight = random.choice(class_weight_options)
        l1_ratio = random.choice(l1_ratio_options) if penalty == "elasticnet" else None
        
        try:
            # We call the CV function with n_jobs=1 because we are already parallelized here
            _, cv_auc, _, _, _, _, _, _ = run_public_clone_l1_cv_noleak(
                full_df,
                min_pos_repertoires=min_pos,
                min_log_odds=min_lo,
                C=C,
                n_splits=N_SPLITS,
                random_state=RANDOM_STATE,
                n_jobs=1,        # FIXED: Inner loop sequential
                use_vj=True,     # FIXED: Explicit True
                penalty_cv=penalty,
                penalty_final=penalty,
                class_weight=class_weight,
                l1_ratio_cv=l1_ratio,
                l1_ratio_final=l1_ratio,
            )
        except Exception as e:
            # Catch errors so one bad trial doesn't kill the batch
            # print(f"Trial {trial_idx} failed: {e}") 
            cv_auc = -1.0
    
        return {
            "trial_idx": trial_idx, "cv_auc": cv_auc,
            "params": {
                "min_pos_repertoires": min_pos, "min_log_odds": min_lo, "C": C,
                "penalty": penalty, "class_weight": class_weight, "l1_ratio": l1_ratio
            }
        }

    print(f"Starting Joblib Random Search for {train_name} using {PARALLEL_JOBS} cores...")
    
    results = Parallel(n_jobs=PARALLEL_JOBS, verbose=1)(
        delayed(run_random_trial)(i) for i in range(N_TRIALS)
    )

    valid_results = [r for r in results if r["cv_auc"] >= 0]
    if not valid_results:
        print(f"All trials failed for {train_name}.")
        continue

    best_result = max(valid_results, key=lambda x: x["cv_auc"])
    best_params = best_result["params"]
    print(f"{train_name} Best AUC: {best_result['cv_auc']:.4f}")
    print(f"Best Params: {best_params}")

    # Retrain final model (can use parallel folds here since outer loop is done)
    final_clf, cv_auc, oof_public, X_pub, y_pub, all_stats, enriched_public, fold_aucs = run_public_clone_l1_cv_noleak(
        full_df,
        min_pos_repertoires=best_params["min_pos_repertoires"],
        min_log_odds=best_params["min_log_odds"],
        C=best_params["C"],
        n_splits=N_SPLITS,
        random_state=RANDOM_STATE,
        n_jobs=PARALLEL_JOBS, # Now we can use parallel cores for the final fit
        use_vj=True,
        penalty_cv=best_params["penalty"],
        penalty_final=best_params["penalty"],
        class_weight=best_params["class_weight"],
        l1_ratio_cv=best_params["l1_ratio"],
        l1_ratio_final=best_params["l1_ratio"],
    )

    with open(log_file, "a") as f:
        f.write(f"{train_name}\t{best_params['min_pos_repertoires']}\t{best_params['min_log_odds']}\t{best_params['C']}\t{best_params['penalty']}\t{best_params['class_weight']}\t{best_params['l1_ratio']}\t{cv_auc}\n")

    # Generate Predictions and Important Sequences
    for test_dir in test_dirs:
        test_name = os.path.basename(test_dir)
        out_path = os.path.join(submission_out_dir, f"{test_name}_submission.tsv")
        if os.path.exists(out_path): continue

        df_test = load_full_dataset(test_dir)
        pred_test_public, X_test_public = predict_public_clone_model(df_test, final_clf=final_clf, enriched_public=enriched_public, use_vj=True)

        if "label_positive" in df_test.columns:
            y_test = df_test[["ID", "label_positive"]].drop_duplicates("ID").set_index("ID")["label_positive"].astype(int).loc[pred_test_public["ID"]].to_numpy()
            try:
                test_auc = roc_auc_score(y_test, pred_test_public["pred_public"])
            except: test_auc = 0.5
            print(f"{test_name} test auc: {test_auc:.4f}")
            with open(log_file, "a") as f: f.write(f"{train_name}\t{test_name}\ttest_auc_public\t{test_auc}\n")

        preds_to_submission(pred_test_public, dataset_name=test_name).to_csv(out_path, sep="\t", index=False)
        print(f"wrote {out_path}")

    if not os.path.exists(imp_out_path):
        n_top = 50000
        all_stats_sorted = all_stats.sort_values("log_odds", ascending=False)
        enriched_ids = set(enriched_public["clone_id"])
        extra_stats = all_stats_sorted[~all_stats_sorted["clone_id"].isin(enriched_ids)].head(max(0, n_top - len(enriched_ids)))
        top_stats = pd.concat([enriched_public, extra_stats], ignore_index=True).drop_duplicates("clone_id")
        stats_to_seq_table(top_stats, dataset_name=train_name, start_rank=1).to_csv(imp_out_path, sep="\t", index=False)
        print(f"wrote {imp_out_path}")

Loading files: 100%|██████████| 400/400 [00:04<00:00, 82.87it/s]


Starting Joblib Random Search for train_dataset_3 using 35 cores...


[Parallel(n_jobs=35)]: Using backend LokyBackend with 35 concurrent workers.
[Parallel(n_jobs=35)]: Done 130 tasks      | elapsed: 69.1min
[Parallel(n_jobs=35)]: Done 400 out of 400 | elapsed: 194.5min finished


train_dataset_3 Best AUC: 0.7920
Best Params: {'min_pos_repertoires': 6, 'min_log_odds': 1.75, 'C': 6.0, 'penalty': 'elasticnet', 'class_weight': 'balanced', 'l1_ratio': 0.85}


Loading files: 100%|██████████| 400/400 [00:04<00:00, 89.95it/s]


wrote results_05E_Pub_Clones/test_dataset_3_submission.tsv
wrote results_05E_Pub_Clones/train_dataset_3_important_sequences.tsv


In [26]:
import os
import pandas as pd
import random
from joblib import Parallel, delayed  
from sklearn.metrics import roc_auc_score

log_file = "training_auc_05E_log.tsv"


pairs = [
    ('train_datasets/train_datasets/train_dataset_6',
     ['test_datasets/test_datasets/test_dataset_6'])
]


min_pos_values      = [5, 6, 7]
min_log_odds_values = [0.9, 1.0, 1.25, 1.5, 1.75]
C_values            = [0.75, 1.0, 2.0, 3.0, 5.0]
penalty_options     = ["l1", "elasticnet"]
class_weight_options = [None, "balanced"]
l1_ratio_options    = [0.15, 0.2, 0.25, 0.5, 0.75, 0.8, 1.0]

#tried N_TRIALS=100, 200,250,300

N_TRIALS = 400
N_SPLITS = 5
RANDOM_STATE = 123

N_FOLD_JOBS = 1        
PARALLEL_JOBS = 35    


submission_out_dir = "results_05E_Pub_Clones"
important_out_dir = "results_05E_Pub_Clones"
os.makedirs(submission_out_dir, exist_ok=True)
os.makedirs(important_out_dir, exist_ok=True)

for train_dir, test_dirs in pairs:
    train_name = os.path.basename(train_dir)
    imp_out_path = os.path.join(important_out_dir, f"{train_name}_important_sequences.tsv")
    submission_paths = [os.path.join(submission_out_dir, f"{os.path.basename(td)}_submission.tsv") for td in test_dirs]

    if os.path.exists(imp_out_path) and all(os.path.exists(p) for p in submission_paths):
        print(f"Skipping {train_name}: all outputs already exist.")
        continue

    full_df = load_full_dataset(train_dir)

    def run_random_trial(trial_idx):
        min_pos = random.choice(min_pos_values)
        min_lo = random.choice(min_log_odds_values)
        C = random.choice(C_values)
        penalty = random.choice(penalty_options)
        class_weight = random.choice(class_weight_options)
        l1_ratio = random.choice(l1_ratio_options) if penalty == "elasticnet" else None
        
        try:
            # We call the CV function with n_jobs=1 because we are already parallelized here
            _, cv_auc, _, _, _, _, _, _ = run_public_clone_l1_cv_noleak(
                full_df,
                min_pos_repertoires=min_pos,
                min_log_odds=min_lo,
                C=C,
                n_splits=N_SPLITS,
                random_state=RANDOM_STATE,
                n_jobs=1,        # FIXED: Inner loop sequential
                use_vj=True,     # FIXED: Explicit True
                penalty_cv=penalty,
                penalty_final=penalty,
                class_weight=class_weight,
                l1_ratio_cv=l1_ratio,
                l1_ratio_final=l1_ratio,
            )
        except Exception as e:
            # Catch errors so one bad trial doesn't kill the batch
            # print(f"Trial {trial_idx} failed: {e}") 
            cv_auc = -1.0
    
        return {
            "trial_idx": trial_idx, "cv_auc": cv_auc,
            "params": {
                "min_pos_repertoires": min_pos, "min_log_odds": min_lo, "C": C,
                "penalty": penalty, "class_weight": class_weight, "l1_ratio": l1_ratio
            }
        }

    print(f"Starting Joblib Random Search for {train_name} using {PARALLEL_JOBS} cores...")
    
    results = Parallel(n_jobs=PARALLEL_JOBS, verbose=1)(
        delayed(run_random_trial)(i) for i in range(N_TRIALS)
    )

    valid_results = [r for r in results if r["cv_auc"] >= 0]
    if not valid_results:
        print(f"All trials failed for {train_name}.")
        continue

    best_result = max(valid_results, key=lambda x: x["cv_auc"])
    best_params = best_result["params"]
    print(f"{train_name} Best AUC: {best_result['cv_auc']:.4f}")
    print(f"Best Params: {best_params}")

    # Retrain final model (can use parallel folds here since outer loop is done)
    final_clf, cv_auc, oof_public, X_pub, y_pub, all_stats, enriched_public, fold_aucs = run_public_clone_l1_cv_noleak(
        full_df,
        min_pos_repertoires=best_params["min_pos_repertoires"],
        min_log_odds=best_params["min_log_odds"],
        C=best_params["C"],
        n_splits=N_SPLITS,
        random_state=RANDOM_STATE,
        n_jobs=PARALLEL_JOBS, # Now we can use parallel cores for the final fit
        use_vj=True,
        penalty_cv=best_params["penalty"],
        penalty_final=best_params["penalty"],
        class_weight=best_params["class_weight"],
        l1_ratio_cv=best_params["l1_ratio"],
        l1_ratio_final=best_params["l1_ratio"],
    )

    with open(log_file, "a") as f:
        f.write(f"{train_name}\t{best_params['min_pos_repertoires']}\t{best_params['min_log_odds']}\t{best_params['C']}\t{best_params['penalty']}\t{best_params['class_weight']}\t{best_params['l1_ratio']}\t{cv_auc}\n")

    # Generate Predictions and Important Sequences
    for test_dir in test_dirs:
        test_name = os.path.basename(test_dir)
        out_path = os.path.join(submission_out_dir, f"{test_name}_submission.tsv")
        if os.path.exists(out_path): continue

        df_test = load_full_dataset(test_dir)
        pred_test_public, X_test_public = predict_public_clone_model(df_test, final_clf=final_clf, enriched_public=enriched_public, use_vj=True)

        if "label_positive" in df_test.columns:
            y_test = df_test[["ID", "label_positive"]].drop_duplicates("ID").set_index("ID")["label_positive"].astype(int).loc[pred_test_public["ID"]].to_numpy()
            try:
                test_auc = roc_auc_score(y_test, pred_test_public["pred_public"])
            except: test_auc = 0.5
            print(f"{test_name} test auc: {test_auc:.4f}")
            with open(log_file, "a") as f: f.write(f"{train_name}\t{test_name}\ttest_auc_public\t{test_auc}\n")

        preds_to_submission(pred_test_public, dataset_name=test_name).to_csv(out_path, sep="\t", index=False)
        print(f"wrote {out_path}")

    if not os.path.exists(imp_out_path):
        n_top = 50000
        all_stats_sorted = all_stats.sort_values("log_odds", ascending=False)
        enriched_ids = set(enriched_public["clone_id"])
        extra_stats = all_stats_sorted[~all_stats_sorted["clone_id"].isin(enriched_ids)].head(max(0, n_top - len(enriched_ids)))
        top_stats = pd.concat([enriched_public, extra_stats], ignore_index=True).drop_duplicates("clone_id")
        stats_to_seq_table(top_stats, dataset_name=train_name, start_rank=1).to_csv(imp_out_path, sep="\t", index=False)
        print(f"wrote {imp_out_path}")

Loading files: 100%|██████████| 400/400 [00:04<00:00, 87.71it/s]


Starting Joblib Random Search for train_dataset_6 using 35 cores...


[Parallel(n_jobs=35)]: Using backend LokyBackend with 35 concurrent workers.
[Parallel(n_jobs=35)]: Done 130 tasks      | elapsed: 70.6min
[Parallel(n_jobs=35)]: Done 400 out of 400 | elapsed: 197.8min finished


train_dataset_6 Best AUC: 1.0000
Best Params: {'min_pos_repertoires': 7, 'min_log_odds': 0.9, 'C': 5.0, 'penalty': 'elasticnet', 'class_weight': None, 'l1_ratio': 0.5}


Loading files: 100%|██████████| 400/400 [00:04<00:00, 88.07it/s]


wrote results_05E_Pub_Clones/test_dataset_6_submission.tsv
wrote results_05E_Pub_Clones/train_dataset_6_important_sequences.tsv


In [27]:
print(x)

NameError: name 'x' is not defined